# 参議院議員選の予測

In [308]:
# モジュールのimport
from pytrends.request import TrendReq
from itertools import zip_longest
from itertools import filterfalse
import pandas as pd
import csv
import pprint

pytrends = TrendReq(hl='ja-JP', tz=360)

# Google trend APIを利用した検索インタレストの取得

In [ ]:
data = pd.read_excel('2016.xlsx')

In [3]:
df = pd.DataFrame(data["キーワードリスト"])

In [5]:
value_list = df.values.tolist()

list3 = []

for list1 in value_list:
    list2 = list1[0].split(',')
    list3.append(list2)

results = []
group_by = 4

In [6]:
for i in range(len(list3)):
    primary_kw = list3[i][0]
    del list3[i][0]
    kw_list = list3[i]
    chunks = zip_longest(*[iter(kw_list)]*group_by)
    p = lambda x: x is None
    
    merged_df = None
    for elems in list(chunks):
        elems = list(filterfalse(p, elems))
        elems.append(primary_kw)
        pytrends.build_payload(elems, cat=0, timeframe='2016-07-01 2016-07-10', geo='JP', gprop='')
        df = pytrends.interest_over_time()
        # 取得結果のスコアは String になる。 float 変換したいので、True/False　が設定されている`isPartial` を削除して float に変換する。   
        del df['isPartial']
        df = df.astype('float64')
        # dataframe　を primary_kw で最大値で正規化する 
        scaled_dataframe = df.div(df[primary_kw].max(), axis=0)
        if merged_df is None:
            merged_df = scaled_dataframe
        else:
            # ValueError: columns overlap but no suffix specified が発生するので、やむなく、'postgresql' を削除
            del scaled_dataframe[primary_kw]
            merged_df = merged_df.join(scaled_dataframe)

    results.append(merged_df.mean()/merged_df.mean().mean())

In [30]:
results

['徳永エリ'    2.446809
 '鉢呂吉雄'    1.106383
 '柿木克弘'    0.936170
 '森英士'     0.191489
 '長谷川岳'    3.000000
 '佐藤和夫'    0.680851
 '中村治'     0.425532
 '飯田佳宏'    0.531915
 '森山佳則'    0.170213
 '水越寛陽'    0.510638
 dtype: float64, '山崎力'      1.556701
 '三國佑貴'     0.000000
 '田名部匡代'    1.443299
 dtype: float64, '田中真一'     1.156463
 '石川幹子'     0.176871
 '木戸口英司'    1.666667
 dtype: float64, '熊谷大'     1.471805
 '油井哲史'    0.248120
 '桜井充'     1.280075
 dtype: float64, '松浦大悟'    0.206107
 '西野晃'     0.137405
 '石井浩郎'    2.656489
 dtype: float64, '月野薫'     1.037143
 '城取良太'    0.231429
 '舟山康江'    1.731429
 dtype: float64, '岩城光英'    1.763359
 '矢内筆勝'    0.229008
 '増子輝彦'    1.007634
 dtype: float64, '郡司彰'       1.244147
 '小林恭子'      1.078595
 ' 武藤優子 '    1.163880
 '石原順子'      1.063545
 '岡田広'       1.369565
 ' 中村幸樹 '    0.080268
 dtype: float64, '田野辺隆男'    0.672794
 '三觜明美'     0.132353
 '上野通子'     2.194853
 dtype: float64, '堀越啓仁'     0.576355
 '安永陽'      0.177340
 '中曽根弘文'    2.246305
 dtype: float64, '大野元裕'     0.39

In [112]:
res = pd.DataFrame(pd.DataFrame(results).sum())
res.columns = [ "trend"]
res["name"] = res.index
res.name = res.name.str.replace("'", "")
res = res.reset_index(drop = True)
res.head()

,trend,name
0,2.446809,徳永エリ
1,1.106383,鉢呂吉雄
2,0.936170,柿木克弘
3,0.191489,森英士
4,3.000000,長谷川岳


# 候補者データの学習(2016)

In [265]:
election = pd.read_csv("2016_cand_ver2.csv")
election.name = election.name.str.replace("　", "")
#election.status = election.where(election.status >= 2, 1)
election = election.drop(["num_votes", "vote_ratio"], axis = 1)
election.sex = (election.sex == "男")
election.head()

,name,elected,status,area,age,sex,party,Recommendation
0,長谷川岳,1,1,1,45,True,自民,1
1,徳永エリ,1,1,1,54,False,民進,1
2,鉢呂吉雄,1,0,1,68,True,民進,1
3,柿木克弘,0,0,1,48,True,自民,1
4,森英士,0,0,1,38,True,共産,0


In [267]:
# 党の整理
election.party = election.party.str.replace("おおさか維新の会", "維新").replace("維新政党新風", "維新")
election.party = election.party.str.replace("こころ", "s").replace("支持政党なし",
                              "s").replace("国民怒りの声", "s").replace("新党改革","s").replace("世界経済共同体党",
                              "s").replace("犬丸勝子と共和党", "s").replace("地球平和党", "s").replace("チャレンジド日本",
                              "s").replace("減税日本", "s").replace("S", "s")
pd.unique(election.party)

array(['自民', '民進', '共産', 's', '無所属', '幸福', '維新', '公明', '社民'], dtype=object)

In [269]:
# 検索トレンドと候補者データのmerge
final = pd.merge(election, res, on = "name", how = "left")
final= final.fillna(0)
final.head()

,name,elected,status,area,age,sex,party,Recommendation,trend
0,長谷川岳,1,1,1,45,True,自民,1,3.000000
1,徳永エリ,1,1,1,54,False,民進,1,2.446809
2,鉢呂吉雄,1,0,1,68,True,民進,1,1.106383
3,柿木克弘,0,0,1,48,True,自民,1,0.936170
4,森英士,0,0,1,38,True,共産,0,0.191489


In [270]:
#　学習用データ（説明変数）の作成
party = pd.get_dummies(final["party"])
party.head()
X = pd.merge(final.drop(['name', 'party','elected', 'area'],axis=1), party, left_index=True, right_index=True)
X.head()

14

In [271]:
# 学習用モジュールインポート
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [323]:
# target
Y = final.elected
# ロジスティック回帰モデルのインスタンスを作成
lr = LogisticRegression()
# ロジスティック回帰モデルの重みを学習
lr.fit(X, Y) 

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [324]:
#学習のスコア表示
lr.score(X, Y)

0.8348214285714286

# 2019年の予測

In [317]:
#データの読み込み
data_19 = pd.read_csv("candidates_19.csv")
data_19.head()

,num,district_J,name,sex,age,party,recommendaton,status
0,0,北海道,森山佳則,m,52,幸福,0,0
1,1,北海道,高橋はるみ,f,65,自民,1,0
2,2,北海道,畠山和也,m,47,共産,0,0
3,3,北海道,岩瀬清次,m,66,労働者,0,0
4,4,北海道,岩本剛人,m,54,自民,1,0


In [318]:
#　党の整理
data_19.party = data_19.party.str.replace("立憲", "民進").replace("国民", "民進")
data_19.party = data_19.party.str.replace("安楽", "s").replace("労働者",
                              "s").replace("町田", "s").replace("高橋","s").replace("N国",
                              "s").replace("加藤", "s").replace("オリーブ", "s").replace("れいわ",
                              "s").replace("日本無党派党", "s")
data_19.head()

,num,district_J,name,sex,age,party,recommendaton,status
0,0,北海道,森山佳則,m,52,幸福,0,0
1,1,北海道,高橋はるみ,f,65,自民,1,0
2,2,北海道,畠山和也,m,47,共産,0,0
3,3,北海道,岩瀬清次,m,66,s,0,0
4,4,北海道,岩本剛人,m,54,自民,1,0


# 検索トレンドの取得

In [309]:
data = pd.read_excel('2019.xlsx')

In [310]:
df = pd.DataFrame(data["キーワードリスト"])

In [311]:
value_list = df.values.tolist()

list3 = []

for list1 in value_list:
    list2 = list1[0].split(',')
    list3.append(list2)

results = []
group_by = 4

In [312]:
for i in range(len(list3)):
    primary_kw = list3[i][0]
    del list3[i][0]
    kw_list = list3[i]
    chunks = zip_longest(*[iter(kw_list)]*group_by)
    p = lambda x: x is None
    
    merged_df = None
    for elems in list(chunks):
        elems = list(filterfalse(p, elems))
        elems.append(primary_kw)
        pytrends.build_payload(elems, cat=0, timeframe='2019-07-10 2019-07-19', geo='JP', gprop='')
        df = pytrends.interest_over_time()
        # 取得結果のスコアは String になる。 float 変換したいので、True/False　が設定されている`isPartial` を削除して float に変換する。   
        # if 'isPartial' in df.columns:
        if df.empty is not True:
            del df['isPartial']
            df = df.astype('float64')
            # dataframe　を primary_kw で最大値で正規化する 
            scaled_dataframe = df.div(df[primary_kw].max(), axis=0)
            if merged_df is None:
                merged_df = scaled_dataframe
            else:
                # ValueError: columns overlap but no suffix specified が発生するので、やむなく、'postgresql' を削除
                del scaled_dataframe[primary_kw]
                merged_df = merged_df.join(scaled_dataframe)
        else:
            print("---------------")
            print("No Data: ", elems)
            print("---------------")

    if merged_df is not None:
        results.append(merged_df.mean()/merged_df.mean().mean())

---------------
No Data:  ["'田辺徹'", "'山田修路'"]
---------------


In [314]:
#　データ整理
res_19 = pd.DataFrame(pd.DataFrame(results).sum())
res_19.columns = [ "trend"]
res_19["name"] = res_19.index
res_19.name = res_19.name.str.replace("'", "")
res_19 = res_19.reset_index(drop = True)
res_19.head()

,trend,name
0,3.129319,高橋はるみ
1,0.994121,畠山和也
2,0.000000,岩瀬清次
3,1.374480,岩本剛人
4,0.501383,森山佳則


In [319]:
#　候補者データと検索トレンドをmerge
final_19 = pd.merge(data_19, res_19, on = "name", how = "left")
final_19= final_19.fillna(0)
final_19.head()

,num,district_J,name,sex,age,party,recommendaton,status,trend
0,0,北海道,森山佳則,m,52,幸福,0,0,0.501383
1,1,北海道,高橋はるみ,f,65,自民,1,0,3.129319
2,2,北海道,畠山和也,m,47,共産,0,0,0.994121
3,3,北海道,岩瀬清次,m,66,s,0,0,0.000000
4,4,北海道,岩本剛人,m,54,自民,1,0,1.374480


In [322]:
# 説明変数の作成
party_19 = pd.get_dummies(final_19["party"])
X_19 = pd.merge(final_19.drop(['name', 'party', 'district_J', 'num'],axis=1), party_19, left_index=True, right_index=True)
X_19.sex = (X_19.sex == "m")
X_19.head()

,sex,age,recommendaton,status,trend,s,公明,共産,幸福,民進,無所属,社民,維新,自民
0,True,52,0,0,0.501383,0,0,0,1,0,0,0,0,0
1,False,65,1,0,3.129319,0,0,0,0,0,0,0,0,1
2,True,47,0,0,0.994121,0,0,1,0,0,0,0,0,0
3,True,66,0,0,0.000000,1,0,0,0,0,0,0,0,0
4,True,54,1,0,1.374480,0,0,0,0,0,0,0,0,1


In [325]:
# 学習モデルを用いた当選確率の予測
prob = lr.predict_proba(X_19)[:, 1]
prob

array([0.158742  , 0.50235483, 0.3621762 , 0.20174061, 0.74296335,
       0.70635213, 0.24673567, 0.37815342, 0.20505149, 0.06710085,
       0.75969601, 0.59558288, 0.25242076, 0.75884715, 0.20369219,
       0.20309021, 0.7613876 , 0.29612055, 0.82600095, 0.12008838,
       0.20490012, 0.57170761, 0.24997479, 0.20429549, 0.52593273,
       0.08597857, 0.23601316, 0.28800128, 0.57050237, 0.60183129,
       0.12614082, 0.76884679, 0.20565778, 0.18877499, 0.04820443,
       0.78260742, 0.29476714, 0.06745056, 0.37684378, 0.33056393,
       0.85009167, 0.55467489, 0.20203994, 0.67937401, 0.81977391,
       0.15074872, 0.06827313, 0.79655594, 0.68705021, 0.78649527,
       0.20084461, 0.12580124, 0.25032349, 0.64301333, 0.49173961,
       0.85346702, 0.91175473, 0.21155181, 0.33931208, 0.68340697,
       0.16665054, 0.3751004 , 0.32879559, 0.20736463, 0.15087743,
       0.61411738, 0.06238836, 0.20024893, 0.20278972, 0.31884302,
       0.21926558, 0.19995159, 0.25007928, 0.75680284, 0.75390